link to original notebook: 
https://deepnote.com/workspace/my-space-d003-e2bb83f7-45c6-44c1-9ca3-2d2fdded6f46/project/Welcome-7a0c0182-8404-4a4a-b0b7-199d3e510618/notebook/Pivotalgorithm-3dc9594cbe624f9781f3fc7dcecc4ce3?utm_content=7a0c0182-8404-4a4a-b0b7-199d3e510618

this notebook implements 2 types of pivot algorithms using CPU, including:
1) the pivot algorithm using atmosphere statistics
2) recursive method.

### 1. Pivot Algorithm with Atmosphere Statistics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

class SAWSimulator:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        
    def generate_initial_saw(self, L):
        """Create straight initial SAW (non-equilibrium starting point)"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk, max_trial):
        """Perform one pivot attempt on a SAW"""
        L = len(walk) - 1
        # try for at most 10 times, return the same walk if failed
        for _ in range(max_trial):
            k = np.random.randint(1, L)  # Random pivot point (can't be endpoint)
            g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
            
            # Apply symmetry operation to subwalk
            subwalk = walk[k:] - walk[k]
            if g == 'rotate90':
                transformed = np.array([[-y,x] for [x,y] in subwalk])
            elif g == 'rotate180':
                transformed = np.array([[-x,-y] for [x,y] in subwalk])
            elif g == 'reflect_x':
                transformed = np.array([[x,-y] for [x,y] in subwalk])
            else:  # reflect_y
                transformed = np.array([[-x,y] for [x,y] in subwalk])
            
            # Check self-avoidance
            new_walk = np.concatenate([walk[:k], walk[k] + transformed])
            if len(np.unique(new_walk, axis=0)) == len(new_walk):
                return new_walk

        return walk
    
    def simulate_atmosphere(self, L, num_samples, max_trial):
        """Estimate atmosphere statistics for walks of length L"""
        atm_counts = []
        walk = self.generate_initial_saw(L)
        
        # Thermalization (burn-in)
        start = time()
        for _ in range(10 * L):
            walk = self.pivot_move(walk, max_trial)
        print(f"Thermalized {10 * L} pivots in {time()-start:.2f}s")
        
        # Production run
        for _ in range(num_samples):
            walk = self.pivot_move(walk, max_trial)
            # Count possible extensions
            last_point = walk[-1]
            count = 0
            for d in self.directions:
                new_point = last_point + d
                if not any(np.all(new_point == walk, axis=1)):
                    count += 1
            atm_counts.append(count)
            
        return np.mean(atm_counts), np.std(atm_counts) / np.sqrt(len(atm_counts))
    
    def estimate_mu(self, num_samples=10000, max_trial=10):
        """Estimate μ using atmosphere statistics"""
        lengths = np.arange(4, self.L_max+1, 2)  # Even lengths to avoid parity effects
        atm_means = []
        atm_stds = []
        
        print("Running SAW simulations for μ estimation...")
        start = time()
        for L in tqdm(lengths):
            mean_atm, std_atm = self.simulate_atmosphere(L, num_samples, max_trial)
            atm_means.append(mean_atm)
            atm_stds.append(std_atm)
            print(f"L={L}: ⟨a⟩ = {mean_atm:.3f} ± {std_atm:.3f}")
        end = time()
        
        # Fit to extract μ and γ
        def model(n, mu, gamma, c):
            return mu * (1 + (gamma-1)/n + c/n**2)
        
        popt, pcov = curve_fit(model, lengths, atm_means)
        mu, gamma = popt[0], popt[1]
        
        # Plot results
        plt.figure(figsize=(10,6))
        plt.scatter(1/lengths, atm_means, label='Simulation Data')
        x_fit = np.linspace(0, 1/lengths[0], 100)
        plt.plot(x_fit, model(1/x_fit, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.xlabel('1/L')
        plt.ylabel('<a>')
        plt.title(f'Atmosphere Statistics for 2D SAWs, N={num_samples}, Time={end-start:.2f}s, μ~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('atmosphere_saw_mu_estimation_cpu.png', dpi=300)
        plt.close()
        
        return mu, gamma

# Benchmarking
if __name__ == "__main__":
    simulator = SAWSimulator(L_max=1000)
    
    start_time = time()
    mu, gamma = simulator.estimate_mu(num_samples=10000, max_trial=10)
    cpu_time = time() - start_time
    
    print(f"CPU Version Results:")
    print(f"Estimated μ = {mu:.6f}")
    print(f"Estimated γ = {gamma:.6f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

Thermalized 100 pivots in 0.02s
  1%|          | 4/499 [00:09<20:38,  2.50s/it]L=10: ⟨a⟩ = 2.754 ± 0.005
Thermalized 120 pivots in 0.03s
  1%|          | 5/499 [00:12<20:27,  2.49s/it]L=12: ⟨a⟩ = 2.731 ± 0.005
Thermalized 140 pivots in 0.05s
  1%|          | 6/499 [00:15<21:43,  2.64s/it]L=14: ⟨a⟩ = 2.720 ± 0.006
Thermalized 160 pivots in 0.04s
  1%|▏         | 7/499 [00:17<21:54,  2.67s/it]L=16: ⟨a⟩ = 2.677 ± 0.006
Thermalized 180 pivots in 0.05s
  2%|▏         | 8/499 [00:20<23:00,  2.81s/it]L=18: ⟨a⟩ = 2.646 ± 0.006
Thermalized 200 pivots in 0.05s
  2%|▏         | 9/499 [00:23<23:16,  2.85s/it]L=20: ⟨a⟩ = 2.685 ± 0.006
Thermalized 220 pivots in 0.07s
  2%|▏         | 10/499 [00:27<24:03,  2.95s/it]L=22: ⟨a⟩ = 2.649 ± 0.006
Thermalized 240 pivots in 0.07s
  2%|▏         | 11/499 [00:30<24:52,  3.06s/it]L=24: ⟨a⟩ = 2.671 ± 0.006
Thermalized 260 pivots in 0.08s
  2%|▏         | 12/499 [00:33<25:36,  3.16s/it]L=26: ⟨a⟩ = 2.666 ± 0.006
Thermalized 280 pivots in 0.07s
  3%|▎         | 13/

### 2. Recursive Method

#### theory behind the recursive method:
$$B(C_{N1}, C_{N2}) = \frac{C_{N1+N2}}{C_{N1} \cdot C_{N2}}$$

#### 1) continuous L sampling approach

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from collections import defaultdict
from scipy.optimize import curve_fit

MU_ESTIMATE = 2.638

class SAWSimulator:
    def __init__(self, L_max=1000):
        self.L_max = L_max
        self.known_counts = {
            1:4, 2:12, 3:36, 4:100, 5:284, 6:780, 7:2172, 8:5916, 9:16268,
            10:44100, 11:120292, 12:324932, 13:881500, 14:2374444, 15:6416596,
            16:17245332, 17:46466676, 18:124658732, 19:335116620, 20:897697164,
            21:2408806028, 22:6444560484, 23:17266613812, 24:46146397316,
            25:123481354908, 26:329712786220, 27:881317491628
        }
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])
        self.B_estimates = defaultdict(list)

    def generate_initial_saw(self, L):
        return np.array([[i, 0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        L = len(walk) - 1
        k = np.random.randint(1, L)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        subwalk = walk[k:] - walk[k]
        transforms = {
            'rotate90': np.array([[-y, x] for [x, y] in subwalk]),
            'rotate180': np.array([[-x, -y] for [x, y] in subwalk]),
            'reflect_x': np.array([[x, -y] for [x, y] in subwalk]),
            'reflect_y': np.array([[-x, y] for [x, y] in subwalk])
        }
        new_walk = np.concatenate([walk[:k], walk[k] + transforms[g]])

        # Check self-avoidance
        return new_walk if len(np.unique(new_walk, axis=0)) == len(new_walk) else walk

    def thermalize(self, walk, L):
        for _ in range(10 * L):
            walk = self.pivot_move(walk)
        return walk

    def recursive_estimate(self, L, num_samples=10000):
        if L in self.known_counts:
            return self.known_counts[L]
            
        L1 = max(k for k in self.known_counts if k <= L//2)
        L2 = L - L1
        c_L2 = self.recursive_estimate(L2, num_samples) if L2 not in self.known_counts else self.known_counts[L2]
        B_est = self.estimate_B(L1, L2, num_samples)
        c_L = B_est * self.known_counts[L1] * c_L2
        
        self.known_counts[L] = c_L
        return c_L

    def estimate_B(self, L1, L2, num_samples):
        B_values = []
        for _ in tqdm(range(num_samples), desc=f"Estimating B({L1},{L2})"):
            walk1 = self.thermalize(self.generate_initial_saw(L1), L1)
            walk2 = self.thermalize(self.generate_initial_saw(L2), L2)
            concatenated = np.concatenate([walk1, walk2[1:] + walk1[-1] - walk2[0] + [1,0]])
            counts = np.bincount([hash(tuple(p)) % 1000000 for p in concatenated])
            B_values.append(1 if np.all(counts <= 1) else 0)
                
        B_est = np.mean(B_values)
        self.B_estimates[(L1, L2)].append(B_est)
        return B_est

    def estimate_mu(self, num_samples=500):
        # Compute counts for all L up to L_max
        start = time()
        for L in range(1, self.L_max + 1):
            if L not in self.known_counts:
                self.recursive_estimate(L, num_samples)
        end = time()
        
        # Prepare data for fitting
        L_list = np.array(sorted(self.known_counts.keys()))
        c_N = np.array([self.known_counts[L] for L in L_list])
        response = np.log(c_N)
        
        # Fitting function
        def fitting_func(N, log_mu, A, gamma):
            return N * log_mu + (gamma - 1) * np.log(N) + A
        
        # Curve fitting
        p0 = [np.log(MU_ESTIMATE), 1.0, 1.344]  # Initial guess: log(mu), A, gamma
        bounds = ([0, -np.inf, 0], [np.inf, np.inf, np.inf])  # Ensure mu, gamma > 0
        popt, _ = curve_fit(fitting_func, L_list, response, p0=p0, bounds=bounds)
        mu = np.exp(popt[0])
        
        # Plotting
        mu_N = [np.exp(np.log(c)/L) for L, c in sorted(self.known_counts.items())]
        
        plt.figure(figsize=(10, 6))
        plt.plot(L_list, mu_N, 'bo-', label='μ_N')
        plt.axhline(y=MU_ESTIMATE, color="k", linestyle="--", label=f"Theoretical μ ≈ {MU_ESTIMATE}")
        plt.xlabel('L')
        plt.ylabel('μ_N')
        plt.title(f'Recursive Method Estimation, N={num_samples}, Time={end-start:.2f}s, μ~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('recursive_mu_vs_l.png')
        plt.close()
        
        return mu
    
# Main execution
if __name__ == "__main__":
    simulator = SAWSimulator(L_max=200)
    start_time = time()
    mu_final = simulator.estimate_mu(num_samples=500)
    cpu_time = time() - start_time
    print(f"\nCPU Recursive Method Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

Estimating B(53,54):  61%|██████    | 304/500 [01:21<00:52,  3.74it/s]


KeyboardInterrupt: 

#### 2) discrete L sampling approach

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from collections import defaultdict
from scipy.optimize import curve_fit

MU_ESTIMATE = 2.638

class SAWSimulator:
    def __init__(self, L_max=1000):
        self.L_max = L_max
        self.known_counts = {
            1: np.log(4), 2: np.log(12), 3: np.log(36), 4: np.log(100), 5: np.log(284),
            6: np.log(780), 7: np.log(2172), 8: np.log(5916), 9: np.log(16268),
            10: np.log(44100), 11: np.log(120292), 12: np.log(324932), 13: np.log(881500),
            14: np.log(2374444), 15: np.log(6416596), 16: np.log(17245332), 17: np.log(46466676),
            18: np.log(124658732), 19: np.log(335116620), 20: np.log(897697164),
            21: np.log(2408806028), 22: np.log(6444560484), 23: np.log(17266613812),
            24: np.log(46146397316), 25: np.log(123481354908), 26: np.log(329712786220),
            27: np.log(881317491628)
        }
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])
        self.B_estimates = defaultdict(list)

    def generate_initial_saw(self, L):
        return np.array([[i, 0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        L = len(walk) - 1
        k = np.random.randint(1, L)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        subwalk = walk[k:] - walk[k]
        transforms = {
            'rotate90': np.array([[-y, x] for [x, y] in subwalk]),
            'rotate180': np.array([[-x, -y] for [x, y] in subwalk]),
            'reflect_x': np.array([[x, -y] for [x, y] in subwalk]),
            'reflect_y': np.array([[-x, y] for [x, y] in subwalk])
        }
        new_walk = np.concatenate([walk[:k], walk[k] + transforms[g]])
        
        # Check self-avoidance
        return new_walk if len(np.unique(new_walk, axis=0)) == len(new_walk) else walk

    def thermalize(self, walk, L):
        for _ in range(10 * L):
            walk = self.pivot_move(walk)
        return walk

    def estimate_B(self, L1, L2, num_samples):
        B_values = []
        for _ in tqdm(range(num_samples), desc=f"Estimating B({L1},{L2})"):
            walk1 = self.thermalize(self.generate_initial_saw(L1), L1)
            walk2 = self.thermalize(self.generate_initial_saw(L2), L2)
            concatenated = np.concatenate([walk1, walk2[1:] + walk1[-1] - walk2[0] + [1,0]])
            counts = np.bincount([hash(tuple(p)) % 1000000 for p in concatenated])
            B_values.append(1 if np.all(counts <= 1) else 0)
                
        B_est = np.mean(B_values)
        B_est = max(B_est, 1e-10)  # Avoid log(0) by setting a minimum value
        self.B_estimates[(L1, L2)].append(B_est)
        return B_est

    def recursive_estimate(self, L, num_samples):
        """Estimate C_L for L = 10, 20, 40, ... using C_{2L} = B * C_L^2 in log space."""
        if L in self.known_counts:
            return self.known_counts[L]
        
        # Compute C_L from C_{L/2}
        L_half = L // 2
        if L % 2 != 0 or L_half < 10:
            raise ValueError(f"L must be a multiple of 2 and at least 20, got L={L}, L/2={L_half}")
        
        # Recursively compute C_{L/2}
        log_c_L_half = self.recursive_estimate(L_half, num_samples)
        
        # Estimate B(L/2, L/2)
        B_est = self.estimate_B(L_half, L_half, num_samples)
        
        # Compute log(C_L) = log(B) + 2 * log(C_{L/2})
        log_c_L = np.log(B_est) + 2 * log_c_L_half
        
        self.known_counts[L] = log_c_L
        return log_c_L

    def estimate_mu(self, num_samples=10000):
        # Compute counts for L = 10, 20, 40, ...
        start = time()
        L = 10
        L_values = []
        while L <= self.L_max:
            L_values.append(L)
            if L not in self.known_counts:
                print(f"Estimating C({L})...")
                self.recursive_estimate(L, num_samples)
            L *= 2
        end = time()
        
        # Prepare data for fitting
        L_list = np.array(L_values)
        log_c_N = np.array([self.known_counts[L] for L in L_list])
        
        # Check for inf/NaN in log_c_N
        valid_indices = np.isfinite(log_c_N)
        L_list = L_list[valid_indices]
        log_c_N = log_c_N[valid_indices]
        if len(L_list) < 3:
            raise ValueError("Not enough valid data points for fitting after removing inf/NaN")
        
        # Fitting function: log(C_L) = L * log(mu) + (gamma - 1) * log(L) + A
        def fitting_func(N, log_mu, A, gamma):
            return N * log_mu + (gamma - 1) * np.log(N) + A
        
        # Curve fitting
        p0 = [np.log(MU_ESTIMATE), 1.0, 1.344]  # Initial guess: log(mu), A, gamma
        bounds = ([0, -np.inf, 0], [np.inf, np.inf, np.inf])
        popt, _ = curve_fit(fitting_func, L_list, log_c_N, p0=p0, bounds=bounds)
        mu_est = np.exp(popt[0])
        
        # Plotting
        mu_N = [np.exp(self.known_counts[L] / L) for L in L_list]
        
        plt.figure(figsize=(10, 6))
        plt.plot(L_list, mu_N, 'bo-', label='μ_N')
        plt.axhline(y=MU_ESTIMATE, color="k", linestyle="--", label=f"Theoretical μ ≈ {MU_ESTIMATE}")
        plt.xlabel('L')
        plt.ylabel('μ_N')
        plt.title(f'Recursive Method Estimation(Sparse), N={num_samples}, Time={end-start:.2f}s, μ~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('recursive_sparse_mu_vs_l.png')
        plt.close()
        
        return mu_est
    
# Main execution
if __name__ == "__main__":
    simulator = SAWSimulator(L_max=10000)
    start_time = time()
    mu_final = simulator.estimate_mu(num_samples=1000)
    cpu_time = time() - start_time
    print(f"\nCPU Recursive Method Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

Estimating C(40)...
Estimating B(20,20): 100%|██████████| 50/50 [00:13<00:00,  3.64it/s]
Estimating C(80)...
Estimating B(40,40): 100%|██████████| 50/50 [00:30<00:00,  1.62it/s]

CPU Recursive Method Results:
Final μ estimate = 2.7672234
Computation Time = 44.94 seconds
